# Problem Statement

Que2.-
In this dataset you have to predict that whether we have to give ATTORNEY to the clients or not (0=Yes, 1=No) based on below factors.
CLMAGE - Age of Client
CLMSEX - Sex of client (0=Male, 1= Female)
SEATBELT - Whether client has Seatbelt or not (0=Yes, 1=No)
CLMINSUR - Whether Client give Insurance or not (0=Yes, 1=No)
LOSS - Loss that client faced

Prepare model by using AdaBoost,CatBoost and XGBoost & Calculate Accuracy by confusion matrix, Calculate accuracy score, precision score, recall score, f1 score.

Dataset : Claimants.csv

### import require Packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Load The DataSet

In [2]:
df = pd.read_csv("claimants.csv")
df.head()

,ATTORNEY,CLMSEX,CLMINSUR,SEATBELT,CLMAGE,LOSS
0,0,0.0,1.0,0.0,50.0,34.940
1,1,1.0,0.0,0.0,18.0,0.891
2,1,0.0,1.0,0.0,5.0,0.330
3,0,0.0,1.0,1.0,31.0,0.037
4,1,0.0,1.0,0.0,30.0,0.038


### EDA

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1340 entries, 0 to 1339
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ATTORNEY  1340 non-null   int64  
 1   CLMSEX    1328 non-null   float64
 2   CLMINSUR  1299 non-null   float64
 3   SEATBELT  1292 non-null   float64
 4   CLMAGE    1151 non-null   float64
 5   LOSS      1340 non-null   float64
dtypes: float64(5), int64(1)
memory usage: 62.9 KB


In [4]:
df.describe()

,ATTORNEY,CLMSEX,CLMINSUR,SEATBELT,CLMAGE,LOSS
count,1340.000000,1328.000000,1299.000000,1292.000000,1151.000000,1340.000000
mean,0.488806,0.558735,0.907621,0.017028,28.414422,3.806307
std,0.500061,0.496725,0.289671,0.129425,20.304451,10.636903
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,1.000000,0.000000,9.000000,0.400000
50%,0.000000,1.000000,1.000000,0.000000,30.000000,1.069500
75%,1.000000,1.000000,1.000000,0.000000,43.000000,3.781500
max,1.000000,1.000000,1.000000,1.000000,95.000000,173.604000


In [5]:
df.shape

(1340, 6)

In [6]:
df.columns

Index(['ATTORNEY', 'CLMSEX', 'CLMINSUR', 'SEATBELT', 'CLMAGE', 'LOSS'], dtype='object')

### Data Cleansing

In [7]:
df.isna().sum()

ATTORNEY      0
CLMSEX       12
CLMINSUR     41
SEATBELT     48
CLMAGE      189
LOSS          0
dtype: int64

In [8]:
df.corr()

,ATTORNEY,CLMSEX,CLMINSUR,SEATBELT,CLMAGE,LOSS
ATTORNEY,1.000000,0.079674,0.079439,-0.057189,0.010680,-0.219715
CLMSEX,0.079674,1.000000,0.066413,-0.015024,-0.023195,-0.027611
CLMINSUR,0.079439,0.066413,1.000000,0.021154,0.049493,0.005821
SEATBELT,-0.057189,-0.015024,0.021154,1.000000,-0.027992,0.111189
CLMAGE,0.010680,-0.023195,0.049493,-0.027992,1.000000,0.064612
LOSS,-0.219715,-0.027611,0.005821,0.111189,0.064612,1.000000


In [9]:
df['CLMAGE'] = df['CLMAGE'].fillna(df['CLMAGE'].mean())
#df['SEATBELT'] = df['SEATBELT'].fillna(df['SEATBELT'].mode(1))
#df['CLMINSUR'] = df['CLMINSUR'].fillna(df['CLMINSUR'].mode(1))
#df['CLMSEX'] = df['CLMSEX'].fillna(df['CLMSEX'].mode(1))


In [10]:
df.isna().sum()
# df.info()

ATTORNEY     0
CLMSEX      12
CLMINSUR    41
SEATBELT    48
CLMAGE       0
LOSS         0
dtype: int64

In [11]:
df.dropna(inplace=True)

In [12]:
df.isna().sum()

ATTORNEY    0
CLMSEX      0
CLMINSUR    0
SEATBELT    0
CLMAGE      0
LOSS        0
dtype: int64

In [13]:
df.shape

(1269, 6)

### Split the dataset into x and y

In [14]:
y = df['ATTORNEY']
print(y.head())
x = df[['CLMSEX', 'CLMINSUR', 'SEATBELT', 'CLMAGE', 'LOSS']]
x.head()

0    0
1    1
2    1
3    0
4    1
Name: ATTORNEY, dtype: int64


,CLMSEX,CLMINSUR,SEATBELT,CLMAGE,LOSS
0,0.0,1.0,0.0,50.0,34.940
1,1.0,0.0,0.0,18.0,0.891
2,0.0,1.0,0.0,5.0,0.330
3,0.0,1.0,1.0,31.0,0.037
4,0.0,1.0,0.0,30.0,0.038


### Split the Dataset into training and testing 

In [15]:
from sklearn.model_selection import train_test_split
X_train, x_test, Y_train, y_test = train_test_split(x, y, train_size=0.7, random_state=123456)


## Built the models

In [16]:
class Models():
    def __init__(self, X_train, Y_train):
        self.X_train = X_train
        self.Y_train = Y_train

    def AdaBoost(self):
        from sklearn.ensemble import AdaBoostClassifier
        model_ada = AdaBoostClassifier().fit(self.X_train, self.Y_train)
        return model_ada

    def Cat_Boost(self):
        from catboost import CatBoostClassifier
        model_cat = CatBoostClassifier().fit(self.X_train, self.Y_train)
        return model_cat

    def Xg_Boost(self):
        from xgboost import XGBClassifier
        model_xg = XGBClassifier().fit(self.X_train, self.Y_train)
        return model_xg


In [17]:
def model_evaluation(model, model_name):
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

    Accuracy = accuracy_score(y_test, model.predict(x_test))
    Precision = precision_score(y_test, model.predict(x_test))
    Recall = recall_score(y_test, model.predict(x_test))
    F1_Score = f1_score(y_test, model.predict(x_test))
    return model_name, Accuracy, Precision, Recall, F1_Score

In [18]:
m1 = Models(X_train, Y_train)

models = [(m1.AdaBoost(),"AdaBoost"),(m1.Cat_Boost(),"CatBoost"),(m1.Xg_Boost(),"XGBoost")]


/home/sunbeam/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Learning rate set to 0.009793
0:	learn: 0.6906084	total: 50.2ms	remaining: 50.1s
1:	learn: 0.6879297	total: 53.4ms	remaining: 26.6s
2:	learn: 0.6853722	total: 55.8ms	remaining: 18.5s
3:	learn: 0.6827540	total: 59.8ms	remaining: 14.9s
4:	learn: 0.6804019	total: 62.3ms	remaining: 12.4s
5:	learn: 0.6779917	total: 65.3ms	remaining: 10.8s
6:	learn: 0.6755577	total: 67.3ms	remaining: 9.55s
7:	learn: 0.6731927	total: 69.3ms	remaining: 8.6s
8:	learn: 0.6711122	total: 71ms	remaining: 7.81s
9:	learn: 0.6690079	total: 72.8ms	remaining: 7.21s
10:	learn: 0.6664864	total: 74.9ms	remaining: 6.73s
11:	learn: 0.6641851	total: 76.6ms	remaining: 6.3s
12:	learn: 0.6621204	total: 77.8ms	remaining: 5.91s
13:	learn: 0.6598039	total: 79.5ms	remaining: 5.6s
14:	learn: 0.6577509	total: 81.3ms	remaining: 5.34s
15:	learn: 0.6555316	total: 83.1ms	remaining: 5.11s
16:	learn: 0.6534329	total: 85ms	remaining: 4.91s
17:	learn: 0.6514877	total: 86.6ms	remaining: 4.73s
18:	learn: 0.6495211	total: 88.4ms	remaining: 4.57s

In [19]:
results = []
for model, model_name in models:
    results.append(model_evaluation(model, model_name))

df1 = pd.DataFrame(results, columns=['Algorithm','Accuracy_Score','Precision_Score','Recall_Score','F1_Score'])
df1 

,Algorithm,Accuracy_Score,Precision_Score,Recall_Score,F1_Score
0,AdaBoost,0.698163,0.745665,0.645,0.691689
1,CatBoost,0.690289,0.744048,0.625,0.679348
2,XGBoost,0.643045,0.679775,0.605,0.640212
